In [80]:
from datasets import load_dataset
import pathlib
import shutil

In [81]:
# def clean_ds_name():
#     path = pathlib.Path("data/town/train/Minowacho")
#     for file in list(path.iterdir()):
#         stem = file.stem.replace(".", "")
#         new_file = file.with_stem(stem)
#         file.rename(new_file.as_posix())

In [82]:
ds = load_dataset("imagefolder", data_dir="data/town")

Resolving data files: 100%|██████████| 1016/1016 [00:00<00:00, 145247.38it/s]
Found cached dataset imagefolder (/home/fummicc1/.cache/huggingface/datasets/imagefolder/default-e54e969aa87ddf66/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
100%|██████████| 1/1 [00:00<00:00, 239.10it/s]


In [83]:
ds["train"][10]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x640>,
 'label': 0}

In [84]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, AutoProcessor

class CLIPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.processor = AutoProcessor.from_pretrained("geolocal/StreetCLIP")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        image = Image.open(item["image_path"]).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        print("text", text)
        inputs = self.processor(
            text,
            images=image,
            return_tensors="pt", 
            padding="max_length",
            max_length=len("Hiyoshihoncho"),
        )
        inputs["input_ids"].squeeze_(dim=0)
        inputs["attention_mask"].squeeze_(dim=0)
        inputs["pixel_values"].squeeze_(dim=0)
        # print("input_ids", inputs["input_ids"].shape)
        # print("attention_mask", inputs["attention_mask"].shape)
        # print("pixel_values", inputs["pixel_values"].shape)        
        return inputs

# Transformations for the image
transform = transforms.Compose([
    transforms.Resize((336, 336)),
    transforms.ToTensor(),
    transforms.Normalize(
        [
            0.48145466,
            0.4578275,
            0.40821073
        ],
        [
            0.26862954,
            0.26130258,
            0.27577711
        ],
    ),
    transforms.ToPILImage(),
])

In [85]:

data = []

folders = list(pathlib.Path("./data/town/train/").iterdir())

for folder in folders:
    folder_name = folder.name
    if folder_name == ".DS_Store":
        continue
    for file in list(folder.iterdir()):
        d = {
            "text": folder_name,
            "image_path": file
        }
        data.append(d)

# Initialize the dataset
dataset = CLIPDataset(data, transform)


In [86]:
from transformers import AutoModel, TrainingArguments, Trainer

# Load the model
model = AutoModel.from_pretrained("geolocal/StreetCLIP")

# Specify the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Create the Trainer and train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


text Hiyoshi
text Minowacho
text Minowacho
text Hiyoshihoncho
text Minowacho
text Minowacho
text Hiyoshi
text Minowacho
text Minowacho
text Hiyoshi
text Hiyoshihoncho
text Minowacho
text Hiyoshihoncho
text Hiyoshi
text Minowacho
text Minowacho


ValueError: The model did not return a loss from the inputs, only the following keys: logits_per_image,logits_per_text,text_embeds,image_embeds,text_model_output,vision_model_output. For reference, the inputs it received are input_ids,attention_mask,pixel_values.